<a href="https://colab.research.google.com/github/hafsahhaudli/Deteksi-Depresi-dengan-LSTM/blob/main/deteksi%20depresi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

!gdown --id 1tAu72jIYVf8j-f6z7oVOCtGviHJiY9nj     # dataset ysyni
!gdown --id 1ZqvYZxIjVsi91AlgHuCkYRmoKkpA1Bqb     # colloquial-indonesian-lexicon
!gdown --id 1-5L1d_j3O-Dfvjwn5S8BVaj-IXRXY9eb     # model cbow
!gdown --id 1wylivefWGV9MSQ70CY5Z07OFAIy1h4ab     # skip-gram

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1tAu72jIYVf8j-f6z7oVOCtGviHJiY9nj
To: /content/ysyni.csv
100% 143k/143k [00:00<00:00, 70.4MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ZqvYZxIjVsi91AlgHuCkYRmoKkpA1Bqb
To: /content/colloquial-indonesian-lexicon.csv
100% 196k/196k [00:00<00:00, 86.3MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Download

In [2]:
import pandas as pd
import re
import string
import nltk
from nltk import word_tokenize, sent_tokenize 
nltk.download('all')
from nltk.corpus import stopwords
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import gensim
from gensim.models import Word2Vec
# df = pd.read_csv('/content/gdrive/My Drive/TUGAS AKHIR/DATASET/ysyni.csv')
df = pd.read_csv('ysyni.csv')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [3]:
df

,Unnamed: 0,0,1,2
0,0,2021-02-23 11:27:17,1364174988434055168,"RT @jusdalle: 🛒 Mal sepi, tenant pd tutup\n🚗 P..."
1,1,2021-02-21 15:27:59,1363510788283977730,Ngebut qada puasa taun kmrn krn hampir sebulan...
2,2,2021-02-15 05:47:51,1361190464758505477,"@tarikulurskuy_ Kalo lagi butuh caffeine kick,..."
3,3,2021-02-10 00:10:41,1359293677437218816,I wonder how's that translator works on the #S...
4,4,2021-02-09 13:59:05,1359139762704027648,"Once a day of ""apa kabar kamu hari ini?"" means..."
...,...,...,...,...
995,995,2018-03-27 04:15:17,978485585676390400,@kokokdirgantoro Twitter is always full of sur...
996,996,2018-03-27 00:55:38,978435342968676352,There are people who believe that it's wrong i...
997,997,2018-03-25 07:47:24,977814190529290240,@sumwand @bagoes_elang @herrysw Bisa kok :) ht...
998,998,2018-03-24 02:29:21,977371761632206848,"@herrysw Ngga om, pulsanya 0. Hanya dipakai un..."


In [4]:
tweet = df['2']

In [5]:
# tweet = tweet[4]

In [6]:
tweet

0      RT @jusdalle: 🛒 Mal sepi, tenant pd tutup\n🚗 P...
1      Ngebut qada puasa taun kmrn krn hampir sebulan...
2      @tarikulurskuy_ Kalo lagi butuh caffeine kick,...
3      I wonder how's that translator works on the #S...
4      Once a day of "apa kabar kamu hari ini?" means...
                             ...                        
995    @kokokdirgantoro Twitter is always full of sur...
996    There are people who believe that it's wrong i...
997    @sumwand @bagoes_elang @herrysw Bisa kok :) ht...
998    @herrysw Ngga om, pulsanya 0. Hanya dipakai un...
999    @herrysw Nomor indosat saya hanya dipakai utk ...
Name: 2, Length: 1000, dtype: object

In [7]:
# proses casefolding
def casefolding(str):
  str = str.lower()
  return str

In [8]:
tw1cs = tweet.apply(casefolding)
# tw1cs = casefolding(tweet)
tw1cs

0      rt @jusdalle: 🛒 mal sepi, tenant pd tutup\n🚗 p...
1      ngebut qada puasa taun kmrn krn hampir sebulan...
2      @tarikulurskuy_ kalo lagi butuh caffeine kick,...
3      i wonder how's that translator works on the #s...
4      once a day of "apa kabar kamu hari ini?" means...
                             ...                        
995    @kokokdirgantoro twitter is always full of sur...
996    there are people who believe that it's wrong i...
997    @sumwand @bagoes_elang @herrysw bisa kok :) ht...
998    @herrysw ngga om, pulsanya 0. hanya dipakai un...
999    @herrysw nomor indosat saya hanya dipakai utk ...
Name: 2, Length: 1000, dtype: object

In [9]:
# proses cleansing
def cleaning(str):
  # menghapus hashtag
  str = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", str)

  # menghapus RT/cc
  str = re.sub('rt', '', str)

  # menghapus karakter twitter atau mention
  str = re.sub(r'(?:@[\w_]+)', ' ', str)

  # menghapus URL
  str = re.sub(r'https?://\S+|www\.\S+', ' ', str)

  #emoji
  str = re.sub(r'["\U0001F600-\U0001F64F" "\U0001F300-\U0001F5FF" "\U0001F680-\U0001F6FF" "\U0001F1E0-\U0001F1FF" "\U00002702-\U000027B0" "\U000024C2-\U0001F251"]', ' ', str)

  # single char
  str = re.sub(r"\b[a-zA-Z]\b", " ", str)

  # menghapus punctuation + angka
  str = str.translate(str.maketrans(' ', ' ', string.punctuation + string.digits))

  # whitespace removal
  str = ' '.join(str.split())     #spasi awal dan akhir
  str = str.strip()               #spasi double di tengah
  str = re.sub('\s+',' ', str)  

  return str

  # # melakukan proses normalize
  # kata_asli = pd.read_csv('/content/gdrive/My Drive/TUGAS AKHIR/DATASET/slang_word_list.csv', sep=',', header=None)
  # text_list = str.split(' ')
  # kamus_slangword = {}

  # for item in kata_asli.values:
  #   kamus_slangword[item[0]] = item[1]

  #   for index in range(len(text_list)):
  #     if text_list[index] in kamus_slangword.keys():
  #       text_list[index] = kamus_slangword[text_list[index]]

  # return ' '.join(text_list)

In [10]:
tw1c = tw1cs.apply(cleaning)
# tw1c = cleaning(tw1cs)
tw1c

0      mal sepi tenant pd tutup penjualan mobil anjlo...
1      ngebut qada puasa taun kmrn krn hampir sebulan...
2      kalo lagi butuh caffeine kick espresso amp ame...
3          wonder how that translator works on the movie
4      once day of apa kabar kamu hari ini means the ...
                             ...                        
995    twitter is always full of surprisingly tweet t...
996    there are people who believe that it wrong ide...
997                                             bisa kok
998    ngga om pulsanya hanya dipakai untuk internet ...
999    nomor indosat saya hanya dipakai utk paket int...
Name: 2, Length: 1000, dtype: object

In [11]:
# proses normalize
def normalize(str):
  # kata_asli = pd.read_csv('/content/gdrive/My Drive/TUGAS AKHIR/DATASET/colloquial-indonesian-lexicon.csv', sep=',', header=None)
  kata_asli = pd.read_csv('colloquial-indonesian-lexicon.csv', sep=',', header=None)
  text_list = str.split(' ')
  kamus_slangword = {}

  for item in kata_asli.values:
    kamus_slangword[item[0]] = item[1]

    for index in range(len(text_list)):
      if text_list[index] in kamus_slangword.keys():
        text_list[index] = kamus_slangword[text_list[index]]

  return ' '.join(text_list)

In [12]:
tw1n = tw1c.apply(normalize)
# tw1n = normalize(tw1c)
tw1n

0      mal sepi banget pada tutup penjualan mobil anj...
1      ngebut qada puasa tahun kemarin karena hampir ...
2      kalo lagi butuh caffeine kick espresso amp ame...
3          wonder how that translator works on the movie
4      once day of apa kabar kamu hari ini means the ...
                             ...                        
995    twitter is always full of surprisingly tweet t...
996    there are people who believe that itu wrong id...
997                                             bisa kok
998    enggak om pulsanya hanya dipakai untuk interne...
999    nomor indosat saya hanya dipakai untuk paket i...
Name: 2, Length: 1000, dtype: object

In [13]:
# kata berulang list
# tw1n = tw1n.apply(lambda x: " ".join([w for i, w in enumerate(x.split()) if x.split().index(w) == i]))
tw1n

0      mal sepi banget pada tutup penjualan mobil anj...
1      ngebut qada puasa tahun kemarin karena hampir ...
2      kalo lagi butuh caffeine kick espresso amp ame...
3          wonder how that translator works on the movie
4      once day of apa kabar kamu hari ini means the ...
                             ...                        
995    twitter is always full of surprisingly tweet t...
996    there are people who believe that itu wrong id...
997                                             bisa kok
998    enggak om pulsanya hanya dipakai untuk interne...
999    nomor indosat saya hanya dipakai untuk paket i...
Name: 2, Length: 1000, dtype: object

In [14]:
def remove_stopword(str):
  str = " ".join([w for i, w in enumerate(str.split()) if str.split().index(w) == i])   # remove kata yang berulang
  tokens = word_tokenize(str)
  listStopword = set(stopwords.words('indonesian'))         # remove stopword b.indo
  
  removed = []
  for t in tokens:                                          # proses tokenisasi
    if t not in listStopword:
      removed.append(t)
  
  return removed

In [15]:
tw1r = tw1n.apply(remove_stopword)
# tw1r = remove_stopword(tw1n)
tw1r

0      [mal, sepi, banget, tutup, penjualan, mobil, a...
1      [ngebut, qada, puasa, kemarin, sebulan, gegara...
2      [kalo, butuh, caffeine, kick, espresso, amp, a...
3      [wonder, how, that, translator, works, on, the...
4      [once, day, of, kabar, means, the, world, for,...
                             ...                        
995    [twitter, is, always, full, of, surprisingly, ...
996    [there, are, people, who, believe, that, wrong...
997                                                   []
998    [om, pulsanya, dipakai, internet, terima, pang...
999    [nomor, indosat, dipakai, paket, internet, ter...
Name: 2, Length: 1000, dtype: object

In [16]:
# # kata berulang per baris
# s = tw1n[1]
# l = s.split()
# k = []
# for i in l:
 
#     # If condition is used to store unique string
#     # in another list 'k'
#     if (s.count(i)>=1 and (i not in k)):
#         k.append(i)
# print(' '.join(k))

In [17]:
# # proses stopword removal

# nltk.download('stopwords')
# nltk.download('punkt')

# def remove_stopword(str):
#   # Melakukan proses stopwords
#   stop_words = set(stopwords.words('/content/gdrive/My Drive/TUGAS AKHIR/DATASET/stopwordID.csv'))
#   kata_tokens = word_tokenize(str)
#   # filter = [w for w in kata_tokens if w not in stop_words]
#   a = []
  
#   # proses tokenizing
#   for w in kata_tokens:
#       if w not in stop_words:
#           a.append(w)
          
#   return a

In [18]:
# proses stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(str):
  return [stemmer.stem(word) for word in str]

In [19]:
tw1s = tw1r.apply(stemming)
# tw1s = stemming(tw1r)
tw1s

0      [mal, sepi, banget, tutup, jual, mobil, anjlok...
1      [ngebut, qada, puasa, kemarin, bulan, gegara, ...
2      [kalo, butuh, caffeine, kick, espresso, amp, a...
3      [wonder, how, that, translator, works, on, the...
4      [once, day, of, kabar, means, the, world, for,...
                             ...                        
995    [twitter, is, always, full, of, surprisingly, ...
996    [there, are, people, who, believe, that, wrong...
997                                                   []
998    [om, pulsa, pakai, internet, terima, panggil, ...
999    [nomor, indosat, pakai, paket, internet, terim...
Name: 2, Length: 1000, dtype: object

In [20]:
# df = pd.DataFrame(tw1s) 
    
# # saving the dataframe 
# df.to_csv('hasil.csv')

In [22]:
tw = tw1s[0] + tw1s[1]
tw

['mal',
 'sepi',
 'banget',
 'tutup',
 'jual',
 'mobil',
 'anjlok',
 'diskon',
 'pajak',
 'wisatawan',
 'asing',
 'turun',
 'gambar',
 'ngebut',
 'qada',
 'puasa',
 'kemarin',
 'bulan',
 'gegara',
 'gerd',
 'oh',
 '']

In [44]:
namaFileModelCbow = "w2vec_wiki_cbow.txt"
namaFileModelSkip = "w2vec_wiki_skip.txt"

modelCbow = gensim.models.KeyedVectors.load_word2vec_format(namaFileModelCbow)
sim = modelCbow.wv.most_similar("mobil")
print("Model CBow : 10 kata terdekat dari mobil:{}".format(sim))

modelSkip = gensim.models.KeyedVectors.load_word2vec_format(namaFileModelSkip)
sim = modelSkip.wv.most_similar("mobil")
print("Model Skip : 10 kata terdekat dari mobil:{}".format(sim))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


Model CBow : 10 kata terdekat dari mobil:[('kendaraan', 0.7141164541244507), ('mobilnya', 0.7094937562942505), ('truk', 0.6933203339576721), ('Mobil', 0.6704035997390747), ('pengemudi', 0.6438370943069458), ('sepeda', 0.6202144622802734), ('minibus', 0.6156409978866577), ('SUV', 0.6140488982200623), ('motor', 0.6114803552627563), ('jeep', 0.599814772605896)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


Model Skip : 10 kata terdekat dari mobil:[('Mobil', 0.7303351163864136), ('mobilnya', 0.7256810069084167), ('truk', 0.7227027416229248), ('kendaraan', 0.7029565572738647), ('minivan', 0.6928724646568298), ('pengemudi', 0.6881382465362549), ('balap', 0.6816605925559998), ('Mayländer', 0.672109842300415), ('mikrovan', 0.6686399579048157), ('kabriolet', 0.6595208644866943)]


In [25]:
# split data train dan test

from sklearn.model_selection import train_test_split

TRAIN_SIZE = 0.8
df_train, df_test = train_test_split(tw1s, test_size=1-TRAIN_SIZE, random_state=42)
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 800
TEST size: 200


In [31]:
import multiprocessing
#TRAIN THIS - DISTANCE,SIMILARITIES, RANK
#Dimensionality- makin banyak makin lama komputasi, makin detail
num_features = 300  #300

min_word_count = 3

num_workers = multiprocessing.cpu_count()

context_size = 7

downsampling = 1e-3

seed = 1

In [32]:
import gensim.models.word2vec as w2v

tex2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers = num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [33]:
tex2vec.build_vocab(tw1s)

In [34]:
print("Anda memiliki panjang vocab: {0}".format(len(tex2vec.wv.vocab)))

Anda memiliki panjang vocab: 569


In [35]:
tex2vec.train(tw1s,total_words=800, epochs=100)

(339804, 704900)

In [37]:
import os

if not os.path.exists('trained'):
    os.makedirs('trained')

In [40]:
import sklearn.manifold
#t sthocastic distributed neighbor embedding or PCA
# video: how to visualize dataset easily
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)
all_word_vector_matrix = modelCbow.trainables.vectors_lockf

AttributeError: ignored

In [ ]:
%%time
#AWAS LAMA BRO
all_word_vector_matrix_2d = tsne.fit_transform(all_word_vector_matrix.reshape((-1, 1)))